In [2]:
import cv2
import numpy as np
from time import sleep
file1='video/video front1.mp4'
file2='video/video front2.mp4'
file3='video/back1.mp4'
file4='video/back2.mp4'
min_width=80 #min width of rectangle
min_height=80 #min height of rectangle
# max_width=250
# max_height=250
offset=6 #Allowed error per pixel  

pos_line=560 #Count line position (default for back1 is 380 and back2 is 390 and 560 for front)

delay= 60 #FPS vídeo

detect = []
cars= 0

	
def get_center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

cap = cv2.VideoCapture(file1)
subtract = cv2.bgsegm.createBackgroundSubtractorMOG()

while True:
    ret , frame1 = cap.read()
    time = float(1/delay)
    sleep(time) 
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey,(3,3),5)
    img_sub = subtract.apply(blur)
    dilate = cv2.dilate(img_sub,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilated = cv2.morphologyEx (dilate, cv2. MORPH_CLOSE , kernel)
    dilated = cv2.morphologyEx (dilated, cv2. MORPH_CLOSE , kernel)
    contour,h=cv2.findContours(dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame1, (0, pos_line), (1200, pos_line), (255,127,0), 3) #frame,starting pos,ending pos,color,thickness
    for(i,c) in enumerate(contour):
        (x,y,w,h) = cv2.boundingRect(c)
        validate_contour= (w >= min_width) and (h >= min_height)
        #   removing the below line which also checks for max height and width  
        #  validate_contour= (((w >= min_width) and (h >= min_height)) and ((w<=max_width) and (h<=max_height)))
        if not validate_contour:
            continue

        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)        
        center = get_center(x, y, w, h)
        detect.append(center)
        cv2.circle(frame1, center, 4, (0, 0,255), -1)

        for (x,y) in detect:
            if y<(pos_line+offset) and y>(pos_line-offset):
                cars+=1
                cv2.line(frame1, (25, pos_line), (1200, pos_line), (0,127,255), 3)  
                detect.remove((x,y))
                print("cars detected : "+str(cars))        
       
    cv2.putText(frame1, "VEHICLE COUNT : "+str(cars), (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),2)
    cv2.imshow("Video Original" , frame1)
    cv2.imshow("Detected",dilated)

    if cv2.waitKey(1) == 27:   #key 27 is escape
        break
    
cv2.destroyAllWindows()
cap.release()


cars detected : 1
cars detected : 2
cars detected : 3
cars detected : 4
cars detected : 5
